In [2]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, matplotlib as mpl, seaborn as sns, warnings

%matplotlib inline
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
plt.rc('font', family='NanumGothic')
rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False

plt.style.use('ggplot')
color_pal = ["#F8766D", "#D39200", "#93AA00", "#00BA38", "#00C19F", "#00B9E3", "#619CFF", "#DB72FB"]

warnings.filterwarnings(action='ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import lightgbm as lgb

In [3]:
seoul = pd.read_csv('./Seoul/seoul_temp.csv')
jeju = pd.read_csv('./Jeju/jeju_temp.csv')
gyeongi = pd.read_csv('./Gyeongi/gyeongi_temp.csv')
others = pd.read_csv('./Other/others_templates.csv')

In [4]:
df = pd.concat([seoul, jeju, gyeongi, others], axis=0, ignore_index=True)

In [5]:
df

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,서울,기타 외국식 음식점업,1.803973e+09
1,202004,서울,기타 주점업,2.313917e+09
2,202004,서울,마사지업,2.219580e+09
3,202004,서울,서양식 음식점업,2.558547e+09
4,202004,서울,여관업,2.119644e+09
...,...,...,...,...
1377,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,7.707144e+09
1378,202007,충북,한식 음식점업,1.149645e+11
1379,202007,충북,호텔업,6.969917e+07
1380,202007,충북,화장품 및 방향제 소매업,2.520107e+09


In [6]:
submission = pd.read_csv('./data/jeju_data_ver1/submission.csv')

In [7]:
submission.columns

Index(['id', 'REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT'], dtype='object')

In [8]:
sub = submission.merge(df,on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'],how='outer').drop(['AMT_x'], axis=1).rename(columns={'AMT_y':'AMT'})

In [9]:
sub.isnull().sum()

id               0
REG_YYMM         0
CARD_SIDO_NM     0
STD_CLSS_NM      0
AMT             12
dtype: int64

# 없는 업종 0값 넣기

In [10]:
df_raw = pd.read_csv('./data/jeju_data_ver1/201901-202003.csv')

In [11]:
df1 = df_raw.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'])['AMT'].sum().reset_index()

In [12]:
sido_list = list(df_raw['CARD_SIDO_NM'].unique())
clss_list = list(df_raw['STD_CLSS_NM'].unique())

In [13]:
dict1 = dict()
for i in sido_list:
    res = df1[df1['CARD_SIDO_NM'] =='{}'.format(i)]['STD_CLSS_NM'].unique()
    result = set(clss_list) - set(res)
    dict1['{}'.format(i)] = list(result)

In [14]:
dict1

{'강원': ['정기 항공 운송업'],
 '경기': ['정기 항공 운송업', '면세점'],
 '경남': ['정기 항공 운송업', '면세점'],
 '경북': ['정기 항공 운송업', '면세점'],
 '광주': ['정기 항공 운송업', '그외 기타 분류안된 오락관련 서비스업', '휴양콘도 운영업', '면세점'],
 '대구': ['정기 항공 운송업', '내항 여객 운송업', '그외 기타 분류안된 오락관련 서비스업', '휴양콘도 운영업'],
 '대전': ['정기 항공 운송업',
  '그외 기타 분류안된 오락관련 서비스업',
  '면세점',
  '버스 운송업',
  '택시 운송업',
  '휴양콘도 운영업',
  '내항 여객 운송업'],
 '부산': ['그외 기타 분류안된 오락관련 서비스업'],
 '서울': ['휴양콘도 운영업', '그외 기타 분류안된 오락관련 서비스업'],
 '세종': ['정기 항공 운송업',
  '호텔업',
  '면세점',
  '버스 운송업',
  '여행사업',
  '택시 운송업',
  '휴양콘도 운영업',
  '내항 여객 운송업',
  '자동차 임대업'],
 '울산': ['정기 항공 운송업', '내항 여객 운송업', '휴양콘도 운영업'],
 '인천': ['휴양콘도 운영업', '그외 기타 분류안된 오락관련 서비스업', '버스 운송업'],
 '전남': ['정기 항공 운송업'],
 '전북': ['그외 기타 분류안된 오락관련 서비스업', '면세점'],
 '제주': ['버스 운송업'],
 '충남': ['면세점'],
 '충북': ['정기 항공 운송업', '그외 기타 분류안된 오락관련 서비스업']}

In [15]:
for idx in range(len(sub)):
    for k,v in dict1.items():
        if sub.iloc[idx,2] == k:
            for clss in v:
                if sub.iloc[idx,3] == clss:
                    sub.iloc[idx,4] = 0

In [16]:
sub[sub['STD_CLSS_NM']=='정기 항공 운송업']

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
30,30,202004,강원,정기 항공 운송업,0.000000e+00
71,71,202004,경기,정기 항공 운송업,0.000000e+00
112,112,202004,경남,정기 항공 운송업,0.000000e+00
153,153,202004,경북,정기 항공 운송업,0.000000e+00
194,194,202004,광주,정기 항공 운송업,0.000000e+00
235,235,202004,대구,정기 항공 운송업,0.000000e+00
276,276,202004,대전,정기 항공 운송업,0.000000e+00
317,317,202004,부산,정기 항공 운송업,2.654896e+09
358,358,202004,서울,정기 항공 운송업,1.381402e+10
399,399,202004,세종,정기 항공 운송업,0.000000e+00


In [17]:
sub.isnull().sum()

id              0
REG_YYMM        0
CARD_SIDO_NM    0
STD_CLSS_NM     0
AMT             2
dtype: int64

In [18]:
sub.fillna(0, inplace=True)

In [19]:
sub.to_csv('./submission_0726.csv', index=False, encoding='utf-8-sig')